In [ ]:
from itertools import permutations
from z3 import Solver, Bool, PbEq, If, RealVal, sat, Implies

def solve24_z3(nums):
    """
    Given a list of 4 numbers (ints or floats), return a list of all
    distinct expressions using +,-,*,/ and parentheses that evaluate to 24.
    """
    solutions = set()

    # all 5 binary-tree shapes on 4 leaves
    shape_names = [
        "((a{}b){}c){}d",
        "(a{}(b{}c)){}d",
        "a{}((b{}c){}d)",
        "a{}(b{}(c{}d))",
        "(a{}b){}(c{}d)"
    ]

    # helper to build a Z3 expression t = x op y
    def apply_op(x, y, op_vars, s):
        p, m, t, d = op_vars  # plus, minus, times, div
        # guard division by zero
        s.add(Implies(d, y != RealVal(0)))
        return If(p, x + y,
               If(m, x - y,
               If(t, x * y,
               x / y)))

    for perm in set(permutations(nums, 4)):
        a0, a1, a2, a3 = perm
        for shape_idx, shape in enumerate(shape_names):
            # create fresh solver
            s = Solver()

            # Boolean selector variables for each operator slot
            op1 = [Bool(f"op1_{shape_idx}_{i}") for i in ["p","m","t","d"]]
            op2 = [Bool(f"op2_{shape_idx}_{i}") for i in ["p","m","t","d"]]
            op3 = [Bool(f"op3_{shape_idx}_{i}") for i in ["p","m","t","d"]]

            # exactly one operator per slot
            s.add(PbEq([(op1[i], 1) for i in range(4)], 1))
            s.add(PbEq([(op2[i], 1) for i in range(4)], 1))
            s.add(PbEq([(op3[i], 1) for i in range(4)], 1))

            # Z3 Reals for our four numbers
            A = RealVal(a0)
            B = RealVal(a1)
            C = RealVal(a2)
            D = RealVal(a3)

            # build the Z3 term t based on shape
            if shape_idx == 0:
                t1 = apply_op(A, B, op1, s)
                t2 = apply_op(t1, C, op2, s)
                t3 = apply_op(t2, D, op3, s)
            elif shape_idx == 1:
                t1 = apply_op(B, C, op2, s)
                t2 = apply_op(A, t1, op1, s)
                t3 = apply_op(t2, D, op3, s)
            elif shape_idx == 2:
                t1 = apply_op(B, C, op2, s)
                t2 = apply_op(t1, D, op3, s)
                t3 = apply_op(A, t2, op1, s)
            elif shape_idx == 3:
                t1 = apply_op(C, D, op3, s)
                t2 = apply_op(B, t1, op2, s)
                t3 = apply_op(A, t2, op1, s)
            else:  # shape_idx == 4
                t1 = apply_op(A, B, op1, s)
                t2 = apply_op(C, D, op3, s)
                t3 = apply_op(t1, t2, op2, s)

            # enforce result == 24
            s.add(t3 == RealVal(24))

            # check satisfiability
            if s.check() == sat:
                m = s.model()

                # recover chosen ops
                chosen = []
                for ops in (op1, op2, op3):
                    for sym, b in zip(['+','-','*','/'], ops):
                        if m[b]:
                            chosen.append(sym)
                            break

                # build a printable string
                expr = shape.format(*chosen)
                expr = expr.replace('a', str(a0)) \
                           .replace('b', str(a1)) \
                           .replace('c', str(a2)) \
                           .replace('d', str(a3))
                solutions.add(expr)

    return sorted(solutions)


In [2]:

puzzle = [3, 3, 8, 8]
sols = solve24_z3(puzzle)
if not sols:
    print("No solution.")
else:
    print(f"Found {len(sols)} solution(s):")
    for s in sols:
        print("  ", s)


Found 1 solution(s):
   8/(3-(8/3))
